---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [63]:
def blight_model():
    import pandas as pd
    import numpy as np
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_curve, auc
    from sklearn.metrics import roc_curve, auc

    ###make train    
    df = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    df_train = df[['ticket_id',
                   'ticket_issued_date',
                   'hearing_date',
                  'violation_code',
                  'fine_amount',
                  'compliance',
                  'discount_amount',
                  'late_fee']]
    ###Make dataframe
    add = pd.read_csv('addresses.csv', encoding = "ISO-8859-1")
    latlon = pd.read_csv('latlons.csv', encoding = "ISO-8859-1")

    df_train = df_train[np.isfinite(df_train ['compliance'])]
    df_train = pd.merge(df_train, add, on='ticket_id', how='left')
    df_train = pd.merge(df_train, latlon, on='address', how='left')
    df_train['ticket_issued_date'] = pd.to_datetime(df_train['ticket_issued_date'], errors = 'coerce')
    df_train['hearing_date'] = pd.to_datetime(df_train['hearing_date'], errors = 'coerce')
    df_train['time_to_hearing'] = (df_train['hearing_date'] - df_train['ticket_issued_date']).dt.days
    df_train = df_train.drop(['ticket_id','address', 'violation_code', 'ticket_issued_date','hearing_date'], axis=1)
    for i in ['lat','lon','time_to_hearing']:
        df_train[i] = df_train = df_train.fillna(df_train[i].mean())

    ###Make Test   
    df = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    test = df[['ticket_id',
                   'ticket_issued_date',
                   'hearing_date',
                  'violation_code',
                  'fine_amount',
                  'discount_amount',
                  'late_fee']]
    ###Make dataframe


    test = pd.merge(test, add, on='ticket_id', how='left')
    test = pd.merge(test, latlon, on='address', how='left')
    test['ticket_issued_date'] = pd.to_datetime(test['ticket_issued_date'], errors = 'coerce')
    test['hearing_date'] = pd.to_datetime(test['hearing_date'], errors = 'coerce')
    test['time_to_hearing'] = (test['hearing_date'] - test['ticket_issued_date']).dt.days
    test = test.drop(['address', 'violation_code', 'ticket_issued_date','hearing_date'], axis=1)
    for i in ['lat','lon','time_to_hearing']:
        test[i] = test = test.fillna(test[i].mean())    


    X = df_train.drop('compliance', axis=1)
    y = df_train['compliance']

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    rate = 0.09
    depth = 12
    clf = GradientBoostingClassifier(learning_rate = rate, max_depth = depth, random_state = 0).fit(X_train, y_train)    
    tid = test['ticket_id']

    X_new = test.drop('ticket_id', axis=1)
    y_new = clf.predict_proba(X_new)
    tid = tid.as_matrix(columns=None)
    answer = pd.DataFrame({'Non-compliant':y_new[:,0],'compliance':y_new[:,1]}, index = tid)

    return answer['compliance']


In [65]:
blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


284932    0.024836
285362    0.009559
285361    0.044931
285338    0.043787
285346    0.043525
285345    0.061376
285347    0.038054
285342    0.421193
285530    0.023000
284989    0.018413
285344    0.020260
285343    0.014783
285340    0.018781
285341    0.082898
285349    0.039275
285348    0.053595
284991    0.018413
285532    0.013606
285406    0.013053
285001    0.011853
285006    0.007573
285405    0.008344
285337    0.012804
285496    0.067187
285497    0.071165
285378    0.299455
285589    0.006351
285585    0.053389
285501    0.112882
285581    0.008706
            ...   
376367    0.018450
376366    0.036062
376362    0.031776
376363    0.056170
376365    0.018450
376364    0.036062
376228    0.778522
376265    0.063890
376286    0.124483
376320    0.032043
376314    0.030050
376327    0.294143
376385    0.242625
376435    0.418537
376370    0.249049
376434    0.038346
376459    0.056731
376478    0.004298
376473    0.027528
376484    0.005493
376482    0.031818
376480    0.

In [17]:
# ###Make dataframe
# import datetime
# df_train = blight_model()
# add = pd.read_csv('addresses.csv', encoding = "ISO-8859-1")
# latlon = pd.read_csv('latlons.csv', encoding = "ISO-8859-1")

# df_train = df_train[np.isfinite(df_train ['compliance'])]
# df_train = pd.merge(df_train, add, on='ticket_id', how='left')
# df_train = pd.merge(df_train, latlon, on='address', how='left')
# df_train['ticket_issued_date'] = pd.to_datetime(df_train['ticket_issued_date'], errors = 'coerce')
# df_train['hearing_date'] = pd.to_datetime(df_train['hearing_date'], errors = 'coerce')
# df_train['time_to_hearing'] = (df_train['hearing_date'] - df_train['ticket_issued_date']).dt.days
# df_train = pd.get_dummies(data = df_train, columns = ['grafitti_status'])
# df_train = df_train.drop(['ticket_id','address', 'violation_code', 'ticket_issued_date','hearing_date'], axis=1)
# for i in ['lat','lon','time_to_hearing']:
#     df_train[i] = df_train = df_train.fillna(df_train[i].mean())
# df_train.head()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,fine_amount,compliance,discount_amount,late_fee,lat,lon,time_to_hearing
0,250.0,0.0,0.0,25.0,42.390729,-83.124268,369.0
1,750.0,1.0,0.0,75.0,42.326937,-83.135118,378.0
2,250.0,0.0,0.0,25.0,42.145257,-83.208233,323.0
3,750.0,0.0,0.0,75.0,42.433466,-83.023493,253.0
4,100.0,0.0,0.0,10.0,42.388641,-83.037858,251.0


In [21]:
# import time
# start_time = time.time()

# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_curve, auc
# from sklearn.metrics import roc_curve, auc

# X = df_train.drop('compliance', axis=1)
# y = df_train['compliance']

# for i in range(5):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=i)
#     rate = 0.09
#     depth = 12


#     clf = GradientBoostingClassifier(learning_rate = rate, max_depth = depth, random_state = i).fit(X_train, y_train)
#     y_score = clf.decision_function(X_test)
#     fpr, tpr, _ = roc_curve(y_test, y_score)
#     roc_auc = auc(fpr, tpr)


#     print ('Rate:', rate, 'Depth:', depth,'R_state:',i, 'AUC:', roc_auc)
#     print('Mins:',(time.time() - start_time)/60)

Rate: 0.09 Depth: 12 R_state: 0 AUC: 0.802746002651
Mins: 2.73268901904424
Rate: 0.09 Depth: 12 R_state: 1 AUC: 0.800434130756
Mins: 5.587049476305643
Rate: 0.09 Depth: 12 R_state: 2 AUC: 0.801290406132
Mins: 8.684713610013326
Rate: 0.09 Depth: 12 R_state: 3 AUC: 0.808784144732
Mins: 12.17603405714035
Rate: 0.09 Depth: 12 R_state: 4 AUC: 0.807709614696
Mins: 15.762187461058298


In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_curve, auc
# import pandas as pd
# import numpy as np

# df = pd.read_csv('train.csv', encoding = "ISO-8859-1")
# df_train = df[['ticket_id',
#                'ticket_issued_date',
#                'hearing_date',
#               'violation_code',
#               'fine_amount',
#               'grafitti_status',
#               'compliance',
#               'discount_amount',
#               'late_fee']]

In [10]:
# feature_importances = pd.DataFrame(clf.feature_importances_,
#                                    index = X_train.columns,
#                                     columns=['importance']).sort_values('importance',                                                                 
#                                                                         ascending=False)

In [12]:
# feature_importances.head(6)

,importance
lon,0.299029
lat,0.284791
time_to_hearing,0.178290
late_fee,0.111867
discount_amount,0.025252
fine_amount,0.023124
